In [1]:
import toolkit

In [2]:
# Define helper functions and import stuff that's needed.

import json

from textwrap import dedent

from hashlib import sha256
from datetime import datetime

from rich.console import Console
from rich.markdown import Markdown
from rich.live import Live
from rich.spinner import Spinner

from IPython.display import clear_output

console = Console(force_jupyter=True, width=100)

def render_markdown(text):
    return Markdown(text)

def render_live(stream):
    response = ""

    with Live(Spinner("dots", text="Loading..."), refresh_per_second=10, console=console) as live:
        for chunk in stream:
            response += chunk
            live.update(Markdown(response))

    return response

class BadParameters(Exception):
    pass

class NoResults(Exception):
    pass

In [3]:
model = 'gemma3:27b'

In [4]:
prompt = toolkit.prompts.function_calling(model)
render_markdown(prompt)

You are a multipurpose, thinking assistant to me, the user :)                                                      

When I send you a message, I will either:                                                                          

 1 start a new thread, or                                                                                          
 2 continue an existing thread.                                                                                    

When I start a new thread, I will ask you something or assign you a task. You must either answer completely or     
complete the task for the thread to end.                                                                           

To respond to my asks or tasks, you must only do one of the following:                                             

a. ask for clarifications b. call one or more functions with some parameters c. conclude the thread with a final   
response.                                                                                                          

Think carefully about what I am asking you. Figure out my intent, and think about what courses of action you can   
take. Try them all and figure out which one works best. You must preface all your responses with the chain of      
thought that lead to your choice and response. Put this thinking at the top of your response, enclosed in a code   
block like so:                                                                                                     

                                                                                                                   
 make sure you mention which option you choose from the above (a, b, c), along with your thoughts here.            
                                                                                                                   

The functions mentioned in choice (b) will be provided in code blocks like so:                                     

                                                                                                                   
 the function specification, in json or yaml format.                                                               
                                                                                                                   

While calling functions, make sure you follow the specification of the function. Do not make assumptions or        
guesses, and feel free to ask the user for clarifications before calling functions.                                

To call a function, you must output the following code block (it must be valid JSON) in your response:             

                                                                                                                   
 {                                                                                                                 
     "id": this string will be mentioned in the function output so you know which function call produced which     
 output,                                                                                                           
     "function": the name of the function to call,                                                                 
     "parameters": a dictionary of parameter names and values to pass to the function                              
 }                                                                                                                 
                                                                                                                   

Note that you must only use the functions provided to you. You may call multiple functions in parallel by including
multiple function_call code blocks in your response. The responses to your function calls might be returned out of 
order, across several user messages.                                                                               

Once you are ready to conclude the thread 

In [5]:
# Write out the function specifications.

specifications = {
    "make_expense_entry": {
        "name": "make_expense_entry",
        "description": "Add an expense entry to the user's personal accounting records.",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {
                    "type": "float",
                    "description": "The amount spent by the user."
                },
                "category": {
                    "type": "string",
                    "enum": ["food", "entertainment", "clothing", "travel", "utilities", "other"],
                    "description": "The category the expense falls under."
                },
                "notes": {
                    "type": "string",
                    "description": "A short note about the expense, that will remind the user of it if needed."
                },
                "date": {
                    "type": "string",
                    "format": "yyyy-mm-dd",
                    "description": "The date of the expense"
                }
            },
            "required": ["amount", "category", "date"]
        },
        "responses": [{
            "type": "string",
            "description": "The ID of the created expense entry."
        }],
        "errors": [{
            "name": "BadEntry",
            "description": "The category or date are invalid."
        }]
    },

    "get_expense_entries": {
        "name": "get_expense_entries",
        "description": "Fetches expense entries from the user's personal accounting records, filtering by category and date.",
        "parameters": {
            "type": "object",
            "properties": {
                "category": {
                    "type": "string",
                    "enum": ["food", "entertainment", "clothing", "travel", "utilities", "other"],
                    "description": "The category the expense falls under, if filtering by category."
                },
                "from_date": {
                    "type": "string",
                    "format": "yyyy-mm-dd",
                    "description": "If filtering by date, this is the start of the search period (inclusive)."
                },
                "to_date": {
                    "type": "string",
                    "format": "yyyy-mm-dd",
                    "description": "If filtering by date, this is the end of the search period (inclusive)."
                }
            },
            "required": []
        },
        "responses": [{
            "type": "object",
            "properties": {
                "entries": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "amount": {
                                "type": "float",
                                "description": "The amount spent by the user."
                            },
                            "category": {
                                "type": "string",
                                "enum": ["food", "entertainment", "clothing", "travel", "utilities", "other"],
                                "description": "The category the expense falls under."
                            },
                            "notes": {
                                "type": "string",
                                "description": "A short note about the expense, that will remind the user of it if needed."
                            },
                            "date": {
                                "type": "string",
                                "format": "yyyy-mm-dd",
                                "description": "The date of the expense"
                            },
                            "created": {
                                "type": "string",
                                "format": "iso8601",
                                "description": "The timestamp recording when the entry was made."
                            },
                            "id": {
                                "type": "string",
                                "description": "The ID of the expense entry."
                            }
                        }
                    }
                },
                "count": {
                    "type": "int",
                    "description": "The number of transactions returned."
                },
                "total": {
                    "type": "float",
                    "description": "The sum total of the amount spent in the transactions returned."
                }
            }
        }],
        "errors": [{
            "name": "BadEntry",
            "description": "The category, from date or to date are invalid."
        }, {
            "name": "NoResults",
            "description": "The search yielded no results with the given filters."
        }]
    },
}

In [6]:
transactions = []

def make_expense_entry(amount: float, category: str, date: str, notes: str = None) -> str:
    global transactions

    try:
        datetime.strptime(date, "%Y-%m-%d")
    except ValueError:
        raise BadParameters("The `date` must be in yyyy-mm-dd format.")

    entry = {
        "amount": amount,
        "category": category,
        "date": date,
        "notes": notes,
        "created": datetime.now().isoformat()
    }

    identifier = sha256(str.encode(json.dumps(entry))).hexdigest()[:8]
    entry["id"] = identifier

    transactions += [entry]

    return identifier

def get_expense_entries(category: str = None, from_date: str = None, to_date: str = None) -> dict:
    global transactions

    filtered = []
    for transaction in transactions:
        if category and transaction["category"].lower() != category.lower(): continue
        if from_date and transaction["date"] < from_date: continue
        if to_date and transaction["date"] > to_date: continue
        filtered.append(transaction)

    if not filtered:
        raise NoResults("The search yielded no results with the given filters.")
    
    return {
        "transactions": filtered,
        "count": len(filtered),
        "total": sum(x["amount"] for x in filtered),
    }

In [7]:
toolkit.functions.register(specifications["make_expense_entry"], make_expense_entry)
toolkit.functions.register(specifications["get_expense_entries"], get_expense_entries)

In [8]:
make_expense_entry(78, "travel", date = "2025-05-19")
make_expense_entry(231.5, "food", date = "2025-05-19")
make_expense_entry(459, "food", date = "2025-05-20")
make_expense_entry(917, "clothing", date = "2025-05-20")

render_markdown("Seeded transactions and registered functions.")

Seeded transactions and registered functions.

In [9]:
functions = toolkit.functions.specify()
render_markdown(functions)

                                                                                                                   
 {                                                                                                                 
   "name": "make_expense_entry",                                                                                   
   "description": "Add an expense entry to the user's personal accounting records.",                               
   "parameters": {                                                                                                 
     "type": "object",                                                                                             
     "properties": {                                                                                               
       "amount": {                                                                                                 
         "type": "float",                                                                                          
         "description": "The amount spent by the user."                                                            
       },                                                                                                          
       "category": {                                                                                               
         "type": "string",                                                                                         
         "enum": [                                                                                                 
           "food",                                                                                                 
           "entertainment",                                                                                        
           "clothing",                                                                                             
           "travel",                                                                                               
           "utilities",                                                                                            
           "other"                                                                                                 
         ],                                                                                                        
         "description": "The category the expense falls under."                                                    
       },                                                                                                          
       "notes": {                                                                                                  
         "type": "string",                                                                                         
         "description": "A short note about the expense, that will remind the user of it if needed."               
       },                                                                                                          
       "date": {                                                                                                   
         "type": "string",                                                                                         
         "format": "yyyy-mm-dd",                                                                                   
         "description": "The date of the expense"                                                                  
       }                                                                                                           
     },                                                                                                            
     "required": [                                                                                                 
       "amount",                                        

In [10]:
task = "Add this to my expenses, and let me know if I'm going over my food budget of 1000 this month."

In [11]:
messages = [
    { "role": "user", "content": prompt },
    { "role": "user", "content": functions },
    { "role": "user", "content": task, "images": ["media/bill.jpg"] }
]

In [12]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({ "role": "assistant", "content": complete_response })

Output()

In [13]:
def respond(result: str) -> None:
    global messages
    messages.append({ "role": "user", "content": result })

In [14]:
calls = toolkit.functions.parse_calls(complete_response)
for call in calls:
    toolkit.functions.execute_call(call, respond)

In [15]:
render_markdown(messages[-1]["content"])

                                                                                                                   
 {                                                                                                                 
     "id": "1",                                                                                                    
     "result": "81791dd0"                                                                                          
 }                                                                                                                 
                                                                                                                   


In [16]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({ "role": "assistant", "content": complete_response })

Output()

In [17]:
calls = toolkit.functions.parse_calls(complete_response)
for call in calls:
    toolkit.functions.execute_call(call, respond)

In [18]:
render_markdown(messages[-1]["content"])

                                                                                                                   
 {                                                                                                                 
     "id": "2",                                                                                                    
     "result": {                                                                                                   
         "transactions": [                                                                                         
             {                                                                                                     
                 "amount": 231.5,                                                                                  
                 "category": "food",                                                                               
                 "date": "2025-05-19",                                                                             
                 "notes": null,                                                                                    
                 "created": "2025-06-15T02:09:21.350766",                                                          
                 "id": "27df6fc1"                                                                                  
             },                                                                                                    
             {                                                                                                     
                 "amount": 459,                                                                                    
                 "category": "food",                                                                               
                 "date": "2025-05-20",                                                                             
                 "notes": null,                                                                                    
                 "created": "2025-06-15T02:09:21.350867",                                                          
                 "id": "f952ca78"                                                                                  
             },                                                                                                    
             {                                                                                                     
                 "amount": 588.0,                                                                                  
                 "category": "food",                                                                               
                 "date": "2025-05-04",                                                                             
                 "notes": "Dinner at TSC",                                                                         
                 "created": "2025-06-15T02:10:18.984603",                                                          
                 "id": "81791dd0"                                                                                  
             }                                                                                                     
         ],                                                                                                        
         "count": 3,                                                                                               
         "total": 1278.5                                                                                           
     }                                                                                                             
 }                                                                                                                 
                                                        

In [19]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({ "role": "assistant", "content": complete_response })

Output()